# Gaussian

In [ ]:
import csv
import os

num_runs = 1



for run in range(1, num_runs + 1):
    print(f"\n================ {run} ================\n")
    
    #########################################################################################################################################################################
    #                                                                           Flow Matching                                                                               #
    #########################################################################################################################################################################
    
    import sys
    sys.path.append('ICLR_Flow_Disentangle')
    
    import numpy as np
    import torch
    import matplotlib.pyplot as plt
    
    from Inference.data import Correction_Gaussian
    from sklearn.ensemble import RandomForestRegressor
    
    
    
    seed = np.random.randint(0, 1000)  
    rho = 0.8
    X_full, y = Correction_Gaussian().generate(1000, rho, seed)

    
    D=X_full.shape[1]
    
    n_jobs=42


    print(X_full.shape)
    
    
    
    
    device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
 
    
    from Flow_Matching.flow_matching_correction import FlowMatchingModel
    import torch
    
    model = FlowMatchingModel(
        X=X_full,
        dim=D,
        device=device,
        hidden_dim=128,        
        time_embed_dim=64,     
        num_blocks=1,
        use_bn=False
    )
    model.fit(num_steps=5000, batch_size=256, lr=1e-3, show_plot=True)

    from scipy.stats import norm
    from Inference.utils import evaluate_importance



    from Inference.estimators_correction import  CPI_Flow_Model_Estimator_correction_new



    estimator = CPI_Flow_Model_Estimator_correction_new(
        regressor =  RandomForestRegressor(
                 n_estimators=500,
                 max_depth=None,
                 min_samples_leaf=5,
                 random_state=seed,
                 n_jobs=n_jobs
                 ),
        flow_model=model
    
    )

    estimator.fit(X_full, y)

    out = estimator.diff_mapped_minus_correction(
        X_full, y,
        retrain="loo",           
        train_kwargs={"num_steps": 5000, "batch_size": 256, "lr": 1e-3, "show_plot": False},
        random_state=42
    )


    ueifs_mapped   = out["ueifs_mapped"]    
    correction_ps  = out["correction_ps"]    
    diff_mapped_ps = out["diff_mapped_ps"]   
    cov_contrib    = out["cov_contrib"]      


    n, d = ueifs_mapped.shape
    assert correction_ps.shape  == (n, d)
    assert diff_mapped_ps.shape == (n, d)
    assert cov_contrib.shape    == (n, d, d)





    mean_ueifs_mapped = ueifs_mapped.mean(axis=0)      
    mean_correction   = correction_ps.mean(axis=0)     
    mean_diff_ps      = diff_mapped_ps.mean(axis=0)    
    std_diff_ps       = diff_mapped_ps.std(axis=0, ddof=1)  
    std_correction    = correction_ps.std(axis=0, ddof=1)   


    check_ps = np.isclose(mean_diff_ps, mean_ueifs_mapped - mean_correction)

    summary = pd.DataFrame({
        "l": np.arange(d),
        "mean_ueifs_mapped": mean_ueifs_mapped,
        "mean_correction_ps": mean_correction,
        "mean_diff_ps":      mean_diff_ps,
        "std_diff_ps":       std_diff_ps,
        "std_correction_ps": std_correction,
        "check_mean_relation": check_ps,  
    })
    print("Column-level summary (first 10):")
    print(summary.head(10))

    from pathlib import Path


    save_path = "./Correction/Gaussian/summary.txt"  


    Path(save_path).parent.mkdir(parents=True, exist_ok=True)


    with open(save_path, "w", encoding="utf-8") as f:
        f.write(summary.to_string(index=False, max_rows=None, max_cols=None))

    print(f"Summary has been saved at: {save_path}")
    

# Gaussian Mixture

In [ ]:
import csv
import os

num_runs = 1



for run in range(1, num_runs + 1):
    print(f"\n {run}  ================\n")
    
    #########################################################################################################################################################################
    #                                                                           Flow Matching                                                                               #
    #########################################################################################################################################################################
    
    import sys
    sys.path.append('ICLR_Flow_Disentangle')
    
    import numpy as np
    import torch
    import matplotlib.pyplot as plt
    
    from Inference.data import Correcrion_Mixture
    from sklearn.ensemble import RandomForestRegressor
    
    
    
    seed = np.random.randint(0, 1000)  
    rho1 = 0.8
    rho2 = 0.2
    mix_weight = 0.2
    X_full, y = Correcrion_Mixture().generate(n=1000, rho1=rho1, rho2=rho2, seed=seed, mix_weight=mix_weight) 

    
    D=X_full.shape[1]
    
    n_jobs=42


    print(X_full.shape)
    
    
    
    
    device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
    
    from Flow_Matching.flow_matching_correction import FlowMatchingModel
    import torch
    
    model = FlowMatchingModel(
        X=X_full,
        dim=D,
        device=device,
        hidden_dim=128,        
        time_embed_dim=64,     
        num_blocks=1,
        use_bn=False
    )
    model.fit(num_steps=5000, batch_size=256, lr=1e-3, show_plot=True)




    from Inference.estimators_correction import  CPI_Flow_Model_Estimator_correction_new



    estimator = CPI_Flow_Model_Estimator_correction_new(
        regressor =  RandomForestRegressor(
                 n_estimators=500,
                 max_depth=None,
                 min_samples_leaf=5,
                 random_state=seed,
                 n_jobs=n_jobs
                 ),
        flow_model=model
    
    )

    estimator.fit(X_full, y)

    out = estimator.diff_mapped_minus_correction(
        X_full, y,
        retrain="loo",           
        train_kwargs={"num_steps": 5000, "batch_size": 256, "lr": 1e-3, "show_plot": False},
        random_state=42
    )

    ueifs_mapped   = out["ueifs_mapped"]     
    correction_ps  = out["correction_ps"]    
    diff_mapped_ps = out["diff_mapped_ps"]   
    cov_contrib    = out["cov_contrib"]      

    n, d = ueifs_mapped.shape
    assert correction_ps.shape  == (n, d)
    assert diff_mapped_ps.shape == (n, d)
    assert cov_contrib.shape    == (n, d, d)





    mean_ueifs_mapped = ueifs_mapped.mean(axis=0)      
    mean_correction   = correction_ps.mean(axis=0)     
    mean_diff_ps      = diff_mapped_ps.mean(axis=0)    
    std_diff_ps       = diff_mapped_ps.std(axis=0, ddof=1)  
    std_correction    = correction_ps.std(axis=0, ddof=1)   


    check_ps = np.isclose(mean_diff_ps, mean_ueifs_mapped - mean_correction)

    summary = pd.DataFrame({
        "l": np.arange(d),
        "mean_ueifs_mapped": mean_ueifs_mapped,
        "mean_correction_ps": mean_correction,
        "mean_diff_ps":      mean_diff_ps,
        "std_diff_ps":       std_diff_ps,
        "std_correction_ps": std_correction,
        "check_mean_relation": check_ps,  
    })
    print("Column-level summary (first 10):")
    print(summary.head(10))

    from pathlib import Path


    save_path = "./Correction/Mixture/summary.txt"  

    Path(save_path).parent.mkdir(parents=True, exist_ok=True)

    with open(save_path, "w", encoding="utf-8") as f:
        f.write(summary.to_string(index=False, max_rows=None, max_cols=None))

    print(f"Summary has been saved at : {save_path}")


# Plot

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import transforms
from pathlib import Path
from typing import Optional


plt.rcParams.update({
    "axes.titlesize": 20,    
    "axes.labelsize": 18,    
    "xtick.labelsize": 16,  
    "ytick.labelsize": 16,   
    "legend.fontsize": 18,   
})

gaussian_path = Path(r"F:\Code\2025\ICLR_Flow_Disentangle\Correction\Gaussian\summary.txt")
mixture_path  = Path(r"F:\Code\2025\ICLR_Flow_Disentangle\Correction\Mixture\summary.txt")

def load_feature_0_9(path: Path):

    df = pd.read_csv(path, sep=r"\s+", engine="python")
    df.columns = [c.strip() for c in df.columns]
    if 'l' in df.columns:
        df = df.sort_values('l')
        df = df[df['l'].between(0, 9)]
        x_labels = df['l'].to_numpy()
    else:
        df = df.iloc[:10].copy()
        x_labels = np.arange(len(df))
    y1 = df['mean_ueifs_mapped'].to_numpy()   
    y2 = df['mean_correction_ps'].to_numpy()  
    return x_labels, y1, y2


x_g, y1_g, y2_g = load_feature_0_9(gaussian_path)
x_m, y1_m, y2_m = load_feature_0_9(mixture_path)


ymax = float(max(y1_g.max(), y2_g.max(), y1_m.max(), y2_m.max()))
y_margin = 0.10 * ymax if ymax > 0 else 1.0  

def plot_panel(ax, x_labels, y1, y2, title: str, ylabel: Optional[str]):
    idx = np.arange(len(x_labels))
    w = 0.4

    b1 = ax.bar(idx - w/2, y1, width=w, label='Naive term')
    b2 = ax.bar(idx + w/2, y2, width=w, label='Correction term')

    c1 = b1.patches[0].get_facecolor()
    c2 = b2.patches[0].get_facecolor()

    ax.set_xticks(idx)
    ax.set_xticklabels(x_labels)
    ax.set_xlabel('Feature')
    if ylabel is not None:
        ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim(0, ymax + y_margin)
    text_offset = transforms.ScaledTranslation(0, 5/72, ax.figure.dpi_scale_trans) 
    for i, v in enumerate(y1):
        ax.text(i - w/2, v, f'{v:.3f}', ha='center', va='bottom',
                transform=ax.transData + text_offset, fontsize=10, color=c1)
    for i, v in enumerate(y2):
        ax.text(i + w/2, v, f'{v:.3f}', ha='center', va='bottom',
                transform=ax.transData + text_offset, fontsize=10, color=c2)


fig = plt.figure(figsize=(16, 6))
gs = GridSpec(nrows=2, ncols=2, height_ratios=[20, 1], hspace=0.25, wspace=0.12, figure=fig)

ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1], sharey=ax1)   
plot_panel(ax1, x_g, y1_g, y2_g, 'Gaussian', ylabel='Value')
plot_panel(ax2, x_m, y1_m, y2_m, 'Gaussian Mixture', ylabel=None)


ax2.tick_params(axis='y', left=False, labelleft=False)


leg_ax = fig.add_subplot(gs[1, :])
leg_ax.axis('off')
handles, labels = ax1.get_legend_handles_labels()
leg_ax.legend(handles, labels, loc='center', ncol=2, frameon=False)


out_pdf = mixture_path.parent / "bars_gaussian_vs_mixture.pdf"
plt.savefig(out_pdf, dpi=200)  
plt.show()

print(f"PDF has been saved at : {out_pdf}")
